Описание бота @VPV_nailsalon_bot:
Данный телеграмм-бот писался на языке Python с использованием импортированных библиотек aiogram и asyncio. Бот обладает следующими возможностями: запись на услуги маникюра (/appointment), оценка салона по различным критериям (/review). К тому же, пользователь может попросить прислать “Прайс-лист” с ценами на услуги (/menu), фотографии работ мастера салона (/works), а также узнать общую информацию о салоне (/about). Бот работает при любом неверно введенном значении и подсказывает пользователю правильный формат ввода. Реплики пользователей, записи клиентов и отзывы с необходимой информацией происходят в файлы “user_id пользователя_replies”, “user_id пользователя_appointments” и “user_id пользователя_reviews” соответственно. 


In [ ]:
from aiogram import Bot, Dispatcher, types, F
import asyncio
from aiogram.filters import CommandStart, Command, or_f
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton, ReplyKeyboardRemove, BotCommand
from aiogram.utils.keyboard import ReplyKeyboardBuilder


# словарь для хранения данных для записи на маникюр
appointment_records = {}

# словарь для хранения данных для отзыва о сервисе
review_records = {}


# множество url-ссылок на фотографии работ
photo_urls = {'https://i.pinimg.com/564x/ee/33/61/ee3361bcbd47791cbe473c01c83af75a.jpg',
              'https://i.pinimg.com/564x/38/ea/a9/38eaa9ccdc966993e187bc2398369ade.jpg',
              'https://i.pinimg.com/564x/c1/46/0e/c1460e1ca43224810f0fb1bafd67db08.jpg',
              'https://i.pinimg.com/564x/85/09/52/850952740f2899ee75a02211d9fd9544.jpg',
              'https://i.pinimg.com/564x/37/56/f1/3756f12cfb71c75f0271778e6b13f474.jpg',
              'https://i.pinimg.com/564x/1c/d4/b6/1cd4b6193006a13277f48980e24aed92.jpg',
              'https://i.pinimg.com/564x/0d/58/b0/0d58b0b8beab94c591f1ce6fc6d84b98.jpg',
              'https://i.pinimg.com/564x/aa/b7/79/aab77960c74139300538fb45a872f01d.jpg',
              'https://i.pinimg.com/736x/52/8d/74/528d74c11361e56bdf3e243c29ebf3c5.jpg'}


# токен телеграмм-бота
TOKEN = '6726045876:AAGTQvCu5UTPAegpAaXVTu3EnkwqfIoHn_k'

# создаем объект бота с указанным токеном
bot = Bot(TOKEN)
# создаем объект диспетчера для обработки входящих сообщений
dp = Dispatcher()


# список команд
async def set_commands(bot: Bot):
    commands = [BotCommand(command='menu', description='Прайс-лист'),
                BotCommand(command='appointment', description='Записаться'),
                BotCommand(command='about', description='О нас'),
                BotCommand(command='works', description='Фотографии работ'),
                BotCommand(command='review', description='Оставить отзыв')]
    await bot.set_my_commands(commands)


# активация команд и запуск бота
async def main():
    await set_commands(bot)
    await dp.start_polling(bot)


# для удаления кнопок после нажатия
del_kbd = ReplyKeyboardRemove()


# кнопки снизу клавиатуры
start_kb2 = ReplyKeyboardBuilder()
start_kb2.add(KeyboardButton(text='Прайс-лист'),
                    KeyboardButton(text='Записаться'),
                    KeyboardButton(text='О нас'),
                    KeyboardButton(text='Фотографии работ'),)
start_kb2.adjust(2, 2)

start_kb3 = ReplyKeyboardBuilder()
start_kb3.attach(start_kb2)
start_kb3.row(KeyboardButton(text='Оставить отзыв'),)


# список услуг на кнопках для записи
services_kb = ReplyKeyboardMarkup(
    keyboard=[
        [
            KeyboardButton(text='Маникюр - покрытие Luxio'),
            KeyboardButton(text='Маникюр - покрытие Luxio + Снятие')
        ],
        [
            KeyboardButton(text='Маникюр - лак'),
            KeyboardButton(text='Японский маникюр'),
            KeyboardButton(text='Детский маникюр')
        ],
    ],
    resize_keyboard=True
)


# функция для сохранения реплик пользователя в файл
def save_message_to_file(user_id, message_text):
    # присваиваем файлу название с уникальным id пользователя
    filename = f"user_{user_id}_replies.txt"
    # записываем текст сообщения в файл с указанием кодировки UTF-8
    with open(filename, "a", encoding="utf-8") as file:
        # каждое сообщение будет записано на отдельной строке
        file.write(message_text + "\n")


# обработка команды /start
@dp.message(CommandStart())
async def start_cmd(message: types.Message):
    user_id = message.from_user.id
    # сохраняем реплику пользователя в файл
    save_message_to_file(user_id, message.text)
    # появление кнопок для действий
    await message.answer('Здравствуйте, я ваш виртуальный помощник! Выберите действие:',
                         reply_markup=start_kb3.as_markup(resize_keyboard=True,
                                                          input_field_placeholder='Выберите действие'))


# обработка команды /menu или кнопки "Прайс-лист"
@dp.message(or_f(Command('menu'), (F.text.lower() == 'прайс-лист')))
async def menu_cmd(message: types.Message):
    user_id = message.from_user.id
    # сохраняем реплику пользователя в файл
    save_message_to_file(user_id, message.text)
    await message.answer('Вот наш прайс-лист:\n'
                         'Маникюр - покрытие Luxio (2000р)\n'
                         'Маникюр - покрытие Luxio + Снятие (2500р)\n'
                         'Маникюр - лак (1000р)\n'
                         'Японский маникюр (1300р)\n'
                         'Детский маникюр (400р)', reply_markup=del_kbd)


# обработка команды /about или кнопки "О нас"
@dp.message(or_f(Command('about'), (F.text.lower() == 'о нас')))
async def about_cmd(message: types.Message):
    user_id = message.from_user.id
    # сохраняем реплику пользователя в файл
    save_message_to_file(user_id, message.text)
    await message.answer('Наш салон, на самом деле, ненастоящий. Это всего лишь проект '
                         'студентки ВШБ Василиевой Полины!', reply_markup=del_kbd)


# обработка команды /works или кнопки "Фотографии работ"
@dp.message(or_f(Command('works'), (F.text.lower() == 'фотографии работ')))
async def works_cmd(message: types.Message):
    user_id = message.from_user.id
    # сохраняем реплику пользователя в файл
    save_message_to_file(user_id, message.text)
    await message.answer('Вот работы нашего салона:', reply_markup=del_kbd)
    # цикл для отправки пользователю фотографий работ
    for url in photo_urls:
        await message.answer_photo(url)


# обработка команды /appointment или кнопки "Записаться"
@dp.message(or_f(F.text.lower() == 'записаться', Command('appointment')))
async def appointment_start(message: types.Message):
    user_id = message.from_user.id
    # сохраняем реплику пользователя в файл
    save_message_to_file(user_id, message.text)
    # создание ключей в словаре для сохранения информации о записи
    appointment_records[user_id] = {'service': None,
                                    'date': None,
                                    'time': None,
                                    'phone': None,
                                    'name': None,
                                    # флаг - начало процесса записи
                                    'in_progress': True}
    # появление кнопок для выбора услуги
    await message.answer('Выберите услугу из списка ниже:', reply_markup=services_kb)


# обработка ввода информации для записи
@dp.message(lambda message: message.text and appointment_records.get(message.from_user.id, {}).get('in_progress'))
async def process_appointment(message: types.Message):
    user_id = message.from_user.id
    # сохраняем реплику пользователя в файл
    save_message_to_file(user_id, message.text)
    # проверяем, что пользователь находится в процессе записи
    if 'in_progress' in appointment_records[user_id] and appointment_records[user_id]['in_progress']:

        # 1 шаг: услуга
        # проверка, что ключ для услуги существует и не имеет значения
        if 'service' in appointment_records[user_id] and appointment_records[user_id]['service'] is None:
            # проверка, что пользователь правильно отправил боту сообщение с выбранной услугой
            if message.text in ["Маникюр - покрытие Luxio", "Маникюр - покрытие Luxio + Снятие",
                                "Маникюр - лак", "Японский маникюр", "Детский маникюр"]:
                service = message.text
                # если все введено верно, то присваиваем ключу service значение выбранной услуги
                appointment_records[user_id]['service'] = service
                # запрашиваем желаемую дату для записи клиента
                await message.answer('Теперь укажите желаемую дату в формате "ДД.ММ"', reply_markup=del_kbd)
            else:
                # в случае неправильного ввода услуги пользователю предложат выбрать ее заново
                await message.answer('Пожалуйста, выберите услугу из списка')

        # 2 шаг: дата
        # проверка, что ключ для даты существует и не имеет значения
        elif 'date' in appointment_records[user_id] and appointment_records[user_id]['date'] is None:
            date = message.text
            # разбиваем введенную строку на день и месяц по точке
            date_parts = date.split('.')
            # проверяем, что введено ровно 2 части
            if len(date_parts) == 2:
                day, month = date_parts
                # проверяем, что обе части состоят из чисел и длина каждой части равна 2
                if day.isdigit() and month.isdigit() and len(day) == 2 and len(month) == 2:
                    # проверяем, что числа находятся в разумных пределах
                    if 1 <= int(day) <= 31 and 1 <= int(month) <= 12:
                        # если все введено верно, то присваиваем ключу date введенное значение
                        appointment_records[user_id]['date'] = date
                        # запрашиваем желаемое время для записи клиента
                        await message.answer('Введите время в формате "ЧЧ:ММ"',
                                             reply_markup=del_kbd)
                    else:
                        # в случае неправильного ввода даты пользователю предложат ввести ее заново
                        await message.answer('Пожалуйста, укажите дату в формате "ДД.ММ"')
                else:
                    await message.answer('Пожалуйста, укажите дату в формате "ДД.ММ"')
            else:
                await message.answer('Пожалуйста, укажите дату в формате "ДД.ММ"')

        # 3 шаг: время
        # проверка, что ключ для времени существует и не имеет значения
        elif 'time' in appointment_records[user_id] and appointment_records[user_id]['time'] is None:
            time = message.text
            # разбиваем введенную строку на часы и минуты по двоеточию
            time_parts = time.split(':')
            # проверяем, что введено ровно 2 части
            if len(time_parts) == 2:
                hours, minutes = time_parts
                # проверяем, что обе части состоят из чисел и что длина каждой части равна 2
                if hours.isdigit() and minutes.isdigit() and len(hours) == 2 and len(minutes) == 2:
                    # проверяем, что числа находятся в разумных пределах
                    if 0 <= int(hours) <= 23 and 0 <= int(minutes) <= 59:
                        # если все введено верно, то присваиваем ключу time введенное значение
                        appointment_records[user_id]['time'] = message.text
                        # запрашиваем номер телефона клиента
                        await message.answer('Укажите ваш номер телефона:', reply_markup=del_kbd)
                    else:
                        # в случае неправильного ввода времени пользователю предложат ввести его заново
                        await message.answer('Пожалуйста, укажите время в формате "ЧЧ:ММ"')
                else:
                    await message.answer('Пожалуйста, укажите время в формате "ЧЧ:ММ"')
            else:
                await message.answer('Пожалуйста, укажите время в формате "ЧЧ:ММ"')

        # 4 шаг: номер телефона
        # проверка, что ключ для номера телефона существует и не имеет значения
        elif 'phone' in appointment_records[user_id] and appointment_records[user_id]['phone'] is None:
            phone = message.text
            # проверяем, что введенные символы являются цифрами
            if phone.isdigit():
                # номер в формате "8..."
                if len(phone) == 11:
                    # если все введено верно, то присваиваем ключу phone введенное значение
                    appointment_records[user_id]['phone'] = phone
                    # запрашиваем имя клиента
                    await message.answer('Теперь укажите ваше имя:', reply_markup=del_kbd)
                else:
                    await message.answer('Пожалуйста, введите номер в формате "+7..." или "8..."')
            # номер в формате "+7..."
            elif phone.startswith('+7') and phone[1:].isdigit() and len(phone) == 12:
                # если все введено верно, то присваиваем ключу phone введенное значение
                appointment_records[user_id]['phone'] = phone
                # запрашиваем имя клиента
                await message.answer('Теперь укажите ваше имя:', reply_markup=del_kbd)
            else:
                # в случае неправильного ввода номера телефона пользователю предложат ввести его заново
                await message.answer('Пожалуйста, введите номер в формате "+7..." или "8..."')

        # 5 шаг: имя клиента
        # проверка, что ключ для имени клиента существует и не имеет значения
        elif 'name' in appointment_records[user_id] and appointment_records[user_id]['name'] is None:
            name = message.text
            # присваиваем ключу name введенное значение
            appointment_records[user_id]['name'] = name
            # отправляем пользователю информацию о записи
            await message.answer(f'{appointment_records[user_id]["name"]}\n'
                                 f'Услуга: {appointment_records[user_id]["service"]}\n'
                                 f'Дата: {appointment_records[user_id]["date"]}\n'
                                 f'Время: {appointment_records[user_id]["time"]}\n'
                                 'Спасибо, что выбрали наш салон! Мы свяжемся с вами позже для подтверждения записи.',
                                 reply_markup=del_kbd)
            # после успешного заполнения всех данных сохраняем запись в файл
            save_appointment_to_file(user_id, appointment_records[user_id])
            # флаг - процесс записи завершен
            appointment_records[user_id]['in_progress'] = False
    else:
        pass


# функция для сохранения данных о записи в файл
def save_appointment_to_file(user_id, appointment_data):
    # определяем имя файла на основе id пользователя
    filename = f"user_{user_id}_appointments.txt"
    # открываем файл на добавление данных в конец
    with open(filename, "a", encoding="utf-8") as file:
        # форматируем данные о записи и записываем их в файл
        file.write(f"Имя: {appointment_data['name']}\n"
                   f"Услуга: {appointment_data['service']}\n"
                   f"Дата: {appointment_data['date']}\n"
                   f"Время: {appointment_data['time']}\n"
                   f"Номер телефона: {appointment_data['phone']}\n\n")


# функция для сохранения данных из отзыва в файл
def save_review_to_file(user_id, review_data):
    # определяем имя файла на основе id пользователя
    filename = f"user_{user_id}_review.txt"
    # открываем файл на добавление данных в конец
    with open(filename, "a", encoding="utf-8") as file:
        # форматируем данные о записи и записываем их в файл
        file.write(f"Работа мастера: {review_data['master']}\n"
                   f"Комфортабельность салона: {review_data['comfort']}\n"
                   f"Доброжелательность администратора: {review_data['administrator']}\n"
                   f"Текст отзыва: {review_data['review']}\n\n")


# обработка команды /review или кнопки "Оставить отзыв"
@dp.message(or_f(F.text.lower() == "оставить отзыв", Command('review')))
async def review_start(message: types.Message):
    user_id = message.from_user.id
    # сохраняем реплику пользователя в файл
    save_message_to_file(user_id, message.text)
    review_records[user_id] = {'master': None,
                               'comfort': None,
                               'administrator': None,
                               'review': None,
                               # флаг - начало процесса оценки
                               'in_progress': True}
    await message.answer('Оставьте впечатление от нашего сервиса! Пожалуйста, оцените работу мастера от 1 до 10:',
                         reply_markup=del_kbd)


# обработка ввода информации для отзыва
@dp.message(lambda message: message.text and review_records.get(message.from_user.id, {}).get('in_progress'))
async def process_review(message: types.Message):
    user_id = message.from_user.id
    # сохраняем реплику пользователя в файл
    save_message_to_file(user_id, message.text)
    # проверяем, что пользователь находится в процессе оставления отзыва
    if 'in_progress' in review_records[user_id] and review_records[user_id]['in_progress']:

        # 1 шаг: оценка мастера
        # проверка, что ключ для оценки мастера существует и не имеет значения
        if 'master' in review_records[user_id] and review_records[user_id]['master'] is None:
            # проверка, что введенная оценка - цифра
            if message.text.isdigit():
                rating = int(message.text)
                # проверка, что введенная оценка находится в заданном диапазоне
                if 1 <= rating <= 10:
                    # если все введено верно, то присваиваем ключу master введенное значение
                    review_records[user_id]['master'] = rating
                    # запрашиваем следующую оценку
                    await message.answer('Оцените комфортабельность салона от 1 до 10:', reply_markup=del_kbd)
                else:
                    # в случае неправильного оценки пользователю предложат ввести ее заново
                    await message.answer('Пожалуйста, введите число от 1 до 10')
            else:
                await message.answer('Пожалуйста, введите число от 1 до 10')

        # 2 шаг: имя комфортабельности салона
        # проверка, что ключ для оценки комфортабельности салона существует и не имеет значения
        elif 'comfort' in review_records[user_id] and review_records[user_id]['comfort'] is None:
            if message.text.isdigit():
                rating = int(message.text)
                if 1 <= rating <= 10:
                    review_records[user_id]['comfort'] = rating
                    await message.answer('Оцените доброжелательность администратора от 1 до 10:',
                                         reply_markup=del_kbd)
                else:
                    await message.answer('Пожалуйста, введите число от 1 до 10')
            else:
                await message.answer('Пожалуйста, введите число от 1 до 10')

        # 3 шаг: оценка работы администратора
        # проверка, что ключ для оценки работы администратора салона существует и не имеет значения
        elif 'administrator' in review_records[user_id] and review_records[user_id]['administrator'] is None:
            if message.text.isdigit():
                rating = int(message.text)
                if 1 <= rating <= 10:
                    review_records[user_id]['administrator'] = rating
                    await message.answer('Теперь оставьте текстовый отзыв:')
                else:
                    await message.answer('Пожалуйста, введите число от 1 до 10')
            else:
                await message.answer('Пожалуйста, введите число от 1 до 10')

        # 4 шаг: ввод текстового отзыва
        # проверка, что ключ текстового отзыва о салоне существует и не имеет значения
        elif 'review' in review_records[user_id] and review_records[user_id]['review'] is None:
            # записываем текстовый отзыв
            review_records[user_id]['review'] = message.text
            # отправляем пользователю информацию о его отзыве
            await message.answer(f"Ваш отзыв:\n"
                                 f'Работа мастера: {review_records[user_id]["master"]}\n'
                                 f'Комфортабельность салона: {review_records[user_id]["comfort"]}\n'
                                 f'Доброжелательность администратора: {review_records[user_id]["administrator"]}\n'
                                 f'Текст отзыва: {review_records[user_id]["review"]}', reply_markup=del_kbd)
            # после успешного заполнения всех данных сохраняем запись в файл
            save_review_to_file(user_id, review_records[user_id])
            # флаг - процесс оценки салона завершен
            review_records[user_id]['in_progress'] = False
    else:
        pass


# обработка всех остальных сообщений
@dp.message()
async def handle_all_messages(message: types.Message):
    user_id = message.from_user.id
    # сохраняем реплику пользователя в файл
    save_message_to_file(user_id, message.text)
    # отправляем сообщение о том, что бот не может обработать данное сообщение
    await message.answer("Простите, я не понял вашего сообщения")


# запуск основной функции асинхронного кода
await main()


In [ ]:
Ссылка на документацию: https://docs.aiogram.dev/en/latest/
aiogram – это фреймворк для создания ботов Telegram на языке Python. Он предоставляет удобные инструменты для взаимодействия с API Telegram, обработки входящих и исходящих сообщений, управления состояниями и многими другими функциями.
Мой бот использует эту библиотеку для отправки и получения сообщений от пользователей, обработки команд и кнопок, а также для других взаимодействий с Telegram.

Ссылка на документацию: https://docs.python.org/3/library/asyncio.html
asyncio – это стандартная библиотека Python для асинхронного программирования. Она предоставляет возможность создавать асинхронные функции и запускать их параллельно, что полезно для работы с сетевыми запросами, вводом/выводом операций и другими асинхронными задачами. 
В моем боте библиотека asyncio используется для создания и управления асинхронными функциями, такими как отправка сообщений и обработка событий.